In [1]:
# Example: Former Subdivision

import pandas as pd

# Merging the same infobox
excel_path = pd.ExcelFile('D:\Kuliah\Skripsi\Ontologi-Sejarah-Indonesia\MichaelDawBalma\Code\infoboxes\infobox-orla.xlsx')
sheet_name = pd.read_excel(excel_path, 'military conflict1')
sheet_name2 = pd.read_excel(excel_path, 'Military Conflict2')
sheet_name3 = pd.read_excel(excel_path, 'military conflict')
csv_file_path = 'military-conflict-final.csv'

merged_df = pd.concat([sheet_name, sheet_name2, sheet_name3])

merged_df.to_csv(csv_file_path, index=False)

In [3]:
# Clean the merged Former Subdivision data
import csv
military_conflict_clean = []

with open(csv_file_path, newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)

    for row in reader:
        military_conflict_clean.append(dict(row))

In [4]:
'''
Searching for longitude and latitude based on the docs:
https://id.wikipedia.org/wiki/Templat:Infobox_former_subdivision (common_name)
'''
import geopy

def get_coordinates(place):
    geolocator = geopy.Nominatim(user_agent='place_coordinates_locator')
    location = geolocator.geocode(place, geometry='wkt')
    if location:
        return location.latitude, location.longitude
    return None,None


for index, value in enumerate(military_conflict_clean):
    latitude, longitude = get_coordinates(value['place'])

    # Add latitude and longitude into the dictionary  
    value['latitude'] = latitude
    value['longitude'] = longitude

In [34]:
'''
Merge column date_start-year_start and date_end-year_end into merged_date_start and merged_date_end
https://raw.githubusercontent.com/w3c/sdw/gh-pages/time/rdf/time.ttl (time:inXSDDate)
'''

month_list = {
    'Januari' : '01',
    'Februari' : '02',
    'Maret' : '03',
    'April' : '04',
    'Mei' : '05',
    'Juni' : '06',
    'Juli' : '07',
    'Agustus' : '08',
    'September' : '09',
    'Oktober' : '10',
    'November' : '11',
    'Desember' : '12',
}

defaultDateStart = '01-01'
defaultDateEnd = '12-31'
defaultYear = 2000

for index, value in enumerate(former_subdivision_clean):
    dateStartUsed = ''
    yearStartUsed = ''
    dateEndUsed = ''
    yearEndUsed = ''

    '''
    Both start and end date will be set to default date if it doesnt have the corresponding data; 
    else it will be on the mm-dd format
    '''
    if value['date_start'] == '' :
        dateStartUsed = defaultDateStart
    else :
        date = value['date_start'].split(" ")
        day = date[0]
        month = month_list[date[1]]
        # change into dd format
        if len(day) == 1 :
            day = '0' + f'{day}'
            # merge day and month into mm-dd format
        dateStartUsed = f'{month}-{day}'
        
    if value['date_end'] == '':
        dateEndUsed = defaultDateEnd
    else :
        date = value['date_end'].split(" ")
        day = date[0]
        month = month_list[date[1]]
        # change into dd format
        if len(day) == 1 :
            day = '0' + f'{day}'
            # merge day and month into mm-dd format
        dateEndUsed = f'{month}-{day}'
    
    yearStartUsed = defaultYear if not value['year_start'] else value['year_start']
    yearEndUsed = defaultYear if not value['year_end'] else value['year_end']
        
    # Merge to yyyy-mm-dd format 
    merged_date_start = f"{yearStartUsed}-{dateStartUsed}"
    merged_date_end = f"{yearEndUsed}-{dateEndUsed}"
    
    value['merged_date_start'] = merged_date_start
    value['merged_date_end'] = merged_date_end


In [35]:
fieldnames = former_subdivision_clean[0].keys()

with open('former-subdivision-final-cleaned.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()

    writer.writerows(former_subdivision_clean)